<a href="https://colab.research.google.com/github/MN-21/handwriting-dnn-features/blob/main/CNN_Baseline_EMNIST_Letters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip install --quiet tensorflow_datasets opencv-python tqdm

import time
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, callbacks, models
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# 1. Load EMNIST Letters from TensorFlow Datasets
(ds_all, ds_info) = tfds.load(
    'emnist/letters',
    split='train+test',
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)  # Combined train+test for custom split

# 2. Convert tf.data to NumPy arrays for stratified split
images, labels = [], []
for img, lbl in tfds.as_numpy(ds_all):
    images.append(img)      # shape (28,28,1)
    labels.append(lbl - 1)  # shift 1–26 → 0–25
images = np.stack(images).astype(np.float32) / 255.0  # normalize to [0,1]
labels = np.array(labels, dtype=int)

# 3. Stratified 80/20 split
X_train, X_test, y_train, y_test = train_test_split(
    images, labels,
    train_size=0.8, test_size=0.2,
    stratify=labels, random_state=42
)  # preserves class balance

# 4. Build tf.data pipelines
BATCH_SIZE = 128
AUTOTUNE   = tf.data.AUTOTUNE

def make_ds(X, y, shuffle=False):
    ds = tf.data.Dataset.from_tensor_slices((X, y))
    if shuffle:
        ds = ds.shuffle(10_000)
    return ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)

train_ds = make_ds(X_train, y_train, shuffle=True)
test_ds  = make_ds(X_test,  y_test)

# 5. Define the CNN with BatchNorm & Dropout
model = models.Sequential([
    layers.Input(shape=(28,28,1)),
    layers.Conv2D(32, 3, padding='same'), layers.BatchNormalization(), layers.Activation('relu'),
    layers.MaxPool2D(),
    layers.Dropout(0.25),

    layers.Conv2D(64, 3, padding='same'), layers.BatchNormalization(), layers.Activation('relu'),
    layers.MaxPool2D(),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(128), layers.BatchNormalization(), layers.Activation('relu'),
    layers.Dropout(0.5),

    layers.Dense(26, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 6. Callbacks: EarlyStopping and ReduceLROnPlateau
es_cb = callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True,
    verbose=1
)  # stops when validation stops improving

rlr_cb = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=1
)  # reduces LR on plateau

# 7. Train for up to 50 epochs
t0 = time.time()
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=50,
    callbacks=[es_cb, rlr_cb],
    verbose=2
)
train_time = time.time() - t0

# 8. Evaluate on test set
t1 = time.time()
test_loss, test_acc = model.evaluate(test_ds, verbose=2)
infer_time = time.time() - t1

# 9. Report
print(f"\nFinal Test Accuracy : {test_acc*100:.2f}%")
print(f"Training Time       : {train_time:.1f}s over {len(history.history['loss'])} epochs")
print(f"Inference Time      : {infer_time:.1f}s on {len(X_test)} samples")


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/emnist/letters/incomplete.GIPBXY_3.1.0/emnist-train.tfrecord*...:   0%|   …

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/emnist/letters/incomplete.GIPBXY_3.1.0/emnist-test.tfrecord*...:   0%|    …

Dataset emnist downloaded and prepared to /root/tensorflow_datasets/emnist/letters/3.1.0. Subsequent calls will reuse this data.
Epoch 1/50
648/648 - 166s - 256ms/step - accuracy: 0.7250 - loss: 0.9316 - val_accuracy: 0.7014 - val_loss: 0.9005 - learning_rate: 1.0000e-03
Epoch 2/50
648/648 - 199s - 307ms/step - accuracy: 0.8396 - loss: 0.5066 - val_accuracy: 0.9056 - val_loss: 0.2988 - learning_rate: 1.0000e-03
Epoch 3/50
648/648 - 154s - 237ms/step - accuracy: 0.8635 - loss: 0.4241 - val_accuracy: 0.9012 - val_loss: 0.2991 - learning_rate: 1.0000e-03
Epoch 4/50
648/648 - 158s - 244ms/step - accuracy: 0.8792 - loss: 0.3744 - val_accuracy: 0.9123 - val_loss: 0.2642 - learning_rate: 1.0000e-03
Epoch 5/50
648/648 - 154s - 237ms/step - accuracy: 0.8897 - loss: 0.3383 - val_accuracy: 0.9225 - val_loss: 0.2303 - learning_rate: 1.0000e-03
Epoch 6/50
648/648 - 158s - 244ms/step - accuracy: 0.8977 - loss: 0.3163 - val_accuracy: 0.9302 - val_loss: 0.2115 - learning_rate: 1.0000e-03
Epoch 7/50
64